## **Federated Learning for attack detection: 5 nodes sharing gradients: Bot-IoT dataset**

IDs from this file = **Bot_cat5_niter_nepoch**, with niter being number of iterations while training, and nepoch number of epochs for local models. 

### Static elements (execute first)

In [1]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') |  (data['proto'] =='igmp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Creating categories dataframe
    data_labels = pd.DataFrame()

    # Drop the invalid features and select interested data features
    data_features=data[['proto','saddr','sport','daddr','dport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    data_features = data_features.fillna(value=-1)
    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['saddr'] = data_features['saddr'].apply(lambda x: x.split(".")[-1])
    data_features['saddr'] = data_features['saddr'].apply(lambda x: x.split(":")[-1])
    data_features['saddr'] = data_features['saddr'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['daddr'] = data_features['daddr'].apply(lambda x: x.split(".")[-1])
    data_features['daddr'] = data_features['daddr'].apply(lambda x: x.split(":")[-1])
    data_features['daddr'] = data_features['daddr'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dport'] = data_features['dport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dport'] = data_features['dport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dport'] = data_features['dport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['saddr'] = data_features['saddr'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['daddr'] = data_features['daddr'].astype(int)
    data_features['dport'] = data_features['dport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)


    # Transform to One Hot Encoding the Categories - normal, dos, reconnaissance, generic, exploits, worms, fuzzers, analysis, backdoor, shellcode
    data_labels.insert(0, 'DoS', data['category'].replace('DoS', 1).replace(['Reconnaissance', 'DDoS', 'Theft'], 0))
    data_labels.insert(1, 'Reconnaissance', data['category'].replace('Reconnaissance', 1).replace([ 'DoS', 'DDoS', 'Theft'], 0))
    data_labels.insert(2, 'DDoS', data['category'].replace('DDoS', 1).replace(['DoS', 'Reconnaissance', 'Theft'], 0))
    data_labels.insert(3, 'Theft', data['category'].replace('Theft', 1).replace(['DoS', 'Reconnaissance', 'DDoS'], 0))
 
    data_labels = pd.get_dummies(data_labels)

    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Value given for the missing columns
    auxCol=0

    # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)


    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [3]:
# Model building and definition
def build_model(input_shape):
    model = models.Sequential()
    model.add(Dense(108, input_shape=input_shape, activation='relu'))
    model.add(layers.Conv2D(filters=16, kernel_size=(1, 2), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(1, 2), padding='same'))
    model.add(layers.Flatten())
    model.add(Dense(108, activation='relu'))
    model.add(Dense(4, activation='softmax'))

    return model 

In [4]:
# Returns values of loss, accuracy, f1, precision and recall of model evaluating with test dataset 
def evaluation(model, x, y): 
    loss, accuracy = model.evaluate(x, y)
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y = np.argmax(y, axis=1)
    report = classification_report(y, y_pred, labels = [i for i in range(4)], target_names=['DoS', 'Reconnaissance', 'DDoS', 'Theft'], output_dict=True)
    # Obtain f1, precision and recall from the report
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    return loss, accuracy, f1, precision, recall

In [5]:
bot1 = pd.read_csv('../datasets/Bot-IoT/cat5_part1.csv')
bot2 = pd.read_csv('../datasets/Bot-IoT/cat5_part2.csv')
bot3 = pd.read_csv('../datasets/Bot-IoT/cat5_part3.csv')
bot4 = pd.read_csv('../datasets/Bot-IoT/cat5_part4.csv')
bot5 = pd.read_csv('../datasets/Bot-IoT/cat5_part5.csv')
test = pd.read_csv('../datasets/Bot-IoT/Bot_test.csv')

In [6]:
bot1['category'].value_counts()

DDoS              1510
DoS               1501
Reconnaissance    1495
Theft              241
Name: category, dtype: int64

In [7]:
x1, y1 = preprocessing(bot1)
x2, y2 = preprocessing(bot2)
x3, y3 = preprocessing(bot3)
x4, y4 = preprocessing(bot4)
x5, y5 = preprocessing(bot5)

In [8]:
# take off 'Normal' samples from test 
test = test[test['category'] != 'Normal']

In [9]:
xtest, ytest = preprocessing(test)

In [10]:
def weiszfeld_update(points, tol=1e-5, max_iter=100):
    """
    Compute the geometric median using Weiszfeld's algorithm.

    Args:
    points: A list of points (numpy arrays) to find the geometric median of.
    tol: Tolerance for stopping criterion.
    max_iter: Maximum number of iterations.

    Returns:
    The geometric median of the points.
    """
    points = np.array(points)
    median = np.mean(points, axis=0)

    for _ in range(max_iter):
        distances = np.linalg.norm(points - median, axis=1)
        nonzero_distances = np.where(distances != 0, distances, np.finfo(float).eps)
        weights = 1 / nonzero_distances
        new_median = np.sum(points * weights[:, None], axis=0) / np.sum(weights)

        if np.linalg.norm(new_median - median) < tol:
            return new_median

        median = new_median

    return median

def aggregate(grad_list):
    """
    Apply Geometric Median aggregation method to a list of gradients.

    Args:
    grad_list: List of gradients from different models. Each element in the list
               is a list of gradients for each layer of a model.

    Returns:
    The geometric median of the gradients.
    """
    # Flatten gradients to compute geometric median
    flat_grads = [tf.concat([tf.reshape(g, [-1]) for g in grad], axis=0).numpy() for grad in grad_list]

    # Compute geometric median using Weiszfeld's algorithm
    flat_median = weiszfeld_update(flat_grads)

    # Reshape the flat median back to the original shape
    median_grad = []
    index = 0
    for grad in grad_list[0]:
        shape = tf.shape(grad)
        size = tf.reduce_prod(shape)
        median_grad.append(tf.reshape(flat_median[index:index + size], shape))
        index += size

    return median_grad

### Experiments

#### niter = 3, nepoch = 5

In [11]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [12]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'Bot_cat5_3_5.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 2 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [13]:
global_model = build_model((24,1,1))

In [14]:
optimizer = keras.optimizers.Adam(learning_rate=1e-4)

In [15]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_test, accuracy_test, f1_test, precision_test, recall_test = evaluation(global_model, xtest, ytest) 

    loss_it.append([loss_test])
    accuracy_it.append([accuracy_test])
    f1_it.append([f1_test])
    precision_it.append([precision_test])
    recall_it.append([recall_test])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/Bot_cat5_3_5.hdf5')

Epoch 1/5
4/4 [==============================] - 2s 607ms/step - loss: 1.7278 - accuracy: 0.0105 - val_loss: 1.4014 - val_accuracy: 0.0074
Epoch 2/5
4/4 [==============================] - 1s 232ms/step - loss: 1.5033 - accuracy: 0.1035 - val_loss: 1.3899 - val_accuracy: 0.0074
Epoch 3/5
4/4 [==============================] - 1s 190ms/step - loss: 1.3360 - accuracy: 0.3526 - val_loss: 1.3789 - val_accuracy: 0.4737
Epoch 4/5
4/4 [==============================] - 1s 141ms/step - loss: 1.1980 - accuracy: 0.5109 - val_loss: 1.3675 - val_accuracy: 0.7442
Epoch 5/5
4/4 [==============================] - 1s 233ms/step - loss: 1.0848 - accuracy: 0.5807 - val_loss: 1.3561 - val_accuracy: 0.8189
Epoch 1/5
4/4 [==============================] - 1s 206ms/step - loss: 1.0224 - accuracy: 0.6155 - val_loss: 1.3414 - val_accuracy: 0.8947
Epoch 2/5
4/4 [==============================] - 0s 87ms/step - loss: 0.9245 - accuracy: 0.6455 - val_loss: 1.3287 - val_accuracy: 0.9074
Epoch 3/5
4/4 [=============

In [16]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[1.1168861389160156], [0.8036187291145325], [0.6332723498344421]]
Accuracy for iterations:  [[0.7098459005355835], [0.7569736242294312], [0.8055024743080139]]
F1 for iterations:  [[0.6908060399261481], [0.7329145024586272], [0.7854514272621329]]
Precision for iterations:  [[0.7173537895367634], [0.7656659674647434], [0.8090604044722649]]
Recall for iterations:  [[0.7098458795057955], [0.7569736339319831], [0.8055024837600305]]


#### niter = 5, nepoch = 5

In [17]:
# Define 
num_nodes = 5
global_updates = 5

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [18]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'Bot_cat5_5_5.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [19]:
global_model = build_model((24,1,1))

In [20]:
optimizer = keras.optimizers.Adam(learning_rate=1e-4)

In [21]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_test, accuracy_test, f1_test, precision_test, recall_test = evaluation(global_model, xtest, ytest) 

    loss_it.append([loss_test])
    accuracy_it.append([accuracy_test])
    f1_it.append([f1_test])
    precision_it.append([precision_test])
    recall_it.append([recall_test])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/Bot_cat5_5_5.hdf5')

Epoch 1/5
4/4 [==============================] - 3s 842ms/step - loss: 1.2323 - accuracy: 0.4517 - val_loss: 1.3783 - val_accuracy: 0.6495
Epoch 2/5
4/4 [==============================] - 1s 206ms/step - loss: 1.1068 - accuracy: 0.4933 - val_loss: 1.3695 - val_accuracy: 0.6800
Epoch 3/5
4/4 [==============================] - 1s 176ms/step - loss: 1.0019 - accuracy: 0.5141 - val_loss: 1.3601 - val_accuracy: 0.6853
Epoch 4/5
4/4 [==============================] - 1s 266ms/step - loss: 0.9151 - accuracy: 0.5465 - val_loss: 1.3506 - val_accuracy: 0.6926
Epoch 5/5
4/4 [==============================] - 1s 265ms/step - loss: 0.8439 - accuracy: 0.5931 - val_loss: 1.3407 - val_accuracy: 0.7000
Epoch 1/5
4/4 [==============================] - 3s 850ms/step - loss: 0.8003 - accuracy: 0.6002 - val_loss: 1.3356 - val_accuracy: 0.6884
Epoch 2/5
4/4 [==============================] - 1s 138ms/step - loss: 0.7444 - accuracy: 0.6479 - val_loss: 1.3239 - val_accuracy: 0.6979
Epoch 3/5
4/4 [============

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
4/4 [==============================] - 3s 785ms/step - loss: 0.5211 - accuracy: 0.6974 - val_loss: 1.0906 - val_accuracy: 0.7463
Epoch 2/5
4/4 [==============================] - 1s 131ms/step - loss: 0.5136 - accuracy: 0.7016 - val_loss: 1.0804 - val_accuracy: 0.7463
Epoch 3/5
4/4 [==============================] - 1s 239ms/step - loss: 0.5074 - accuracy: 0.7153 - val_loss: 1.0706 - val_accuracy: 0.7463
Epoch 4/5
4/4 [==============================] - 1s 146ms/step - loss: 0.5027 - accuracy: 0.7272 - val_loss: 1.0609 - val_accuracy: 0.7463
Epoch 5/5
4/4 [==============================] - 1s 232ms/step - loss: 0.4994 - accuracy: 0.7161 - val_loss: 1.0516 - val_accuracy: 0.7463
Epoch 1/5
4/4 [==============================] - 2s 508ms/step - loss: 0.5097 - accuracy: 0.7316 - val_loss: 1.0603 - val_accuracy: 0.7463
Epoch 2/5
4/4 [==============================] - 1s 364ms/step - loss: 0.5056 - accuracy: 0.7295 - val_loss: 1.0493 - val_accuracy: 0.7463
Epoch 3/5
4/4 [============

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
4/4 [==============================] - 2s 529ms/step - loss: 0.4717 - accuracy: 0.7411 - val_loss: 0.8541 - val_accuracy: 0.7463
Epoch 2/5
4/4 [==============================] - 1s 144ms/step - loss: 0.4638 - accuracy: 0.7722 - val_loss: 0.8524 - val_accuracy: 0.7463
Epoch 3/5
4/4 [==============================] - 1s 140ms/step - loss: 0.4610 - accuracy: 0.8035 - val_loss: 0.8508 - val_accuracy: 0.7463
Epoch 4/5
4/4 [==============================] - 1s 141ms/step - loss: 0.4576 - accuracy: 0.8043 - val_loss: 0.8488 - val_accuracy: 0.7463
Epoch 5/5
4/4 [==============================] - 1s 130ms/step - loss: 0.4563 - accuracy: 0.8051 - val_loss: 0.8471 - val_accuracy: 0.7463
Epoch 1/5
4/4 [==============================] - 2s 557ms/step - loss: 0.4676 - accuracy: 0.8075 - val_loss: 0.8943 - val_accuracy: 0.7463
Epoch 2/5
4/4 [==============================] - 1s 136ms/step - loss: 0.4628 - accuracy: 0.7998 - val_loss: 0.8911 - val_accuracy: 0.7463
Epoch 3/5
4/4 [============

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
4/4 [==============================] - 4s 877ms/step - loss: 0.4448 - accuracy: 0.7867 - val_loss: 0.9061 - val_accuracy: 0.7463
Epoch 2/5
4/4 [==============================] - 1s 160ms/step - loss: 0.4377 - accuracy: 0.7967 - val_loss: 0.9203 - val_accuracy: 0.7463
Epoch 3/5
4/4 [==============================] - 0s 106ms/step - loss: 0.4341 - accuracy: 0.8259 - val_loss: 0.9364 - val_accuracy: 0.7463
Epoch 4/5
4/4 [==============================] - 0s 106ms/step - loss: 0.4331 - accuracy: 0.8270 - val_loss: 0.9536 - val_accuracy: 0.7463
Epoch 5/5
4/4 [==============================] - 0s 106ms/step - loss: 0.4301 - accuracy: 0.8257 - val_loss: 0.9710 - val_accuracy: 0.7463
Epoch 1/5
4/4 [==============================] - 3s 714ms/step - loss: 0.4423 - accuracy: 0.8217 - val_loss: 1.0990 - val_accuracy: 0.7463
Epoch 2/5
4/4 [==============================] - 0s 110ms/step - loss: 0.4381 - accuracy: 0.8241 - val_loss: 1.1165 - val_accuracy: 0.7463
Epoch 3/5
4/4 [============

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
4/4 [==============================] - 3s 646ms/step - loss: 0.4165 - accuracy: 0.8264 - val_loss: 1.4538 - val_accuracy: 0.7463
Epoch 2/5
4/4 [==============================] - 1s 166ms/step - loss: 0.4118 - accuracy: 0.8178 - val_loss: 1.4874 - val_accuracy: 0.7463
Epoch 3/5
4/4 [==============================] - 0s 117ms/step - loss: 0.4074 - accuracy: 0.8470 - val_loss: 1.5239 - val_accuracy: 0.7463
Epoch 4/5
4/4 [==============================] - 1s 133ms/step - loss: 0.4072 - accuracy: 0.8462 - val_loss: 1.5616 - val_accuracy: 0.7463
Epoch 5/5
4/4 [==============================] - 1s 256ms/step - loss: 0.4039 - accuracy: 0.8451 - val_loss: 1.6005 - val_accuracy: 0.7463
Epoch 1/5
4/4 [==============================] - 2s 426ms/step - loss: 0.4155 - accuracy: 0.8404 - val_loss: 1.8512 - val_accuracy: 0.7463
Epoch 2/5
4/4 [==============================] - 1s 237ms/step - loss: 0.4105 - accuracy: 0.8444 - val_loss: 1.8849 - val_accuracy: 0.7463
Epoch 3/5
4/4 [============

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[1.0944833755493164], [0.799087643623352], [0.6344261765480042], [0.662285327911377], [0.8050647377967834]]
Accuracy for iterations:  [[0.7168513536453247], [0.7930200099945068], [0.8075404167175293], [0.8209145069122314], [0.8226977586746216]]
F1 for iterations:  [[0.6933587836607265], [0.7705028889939272], [0.7868871021900775], [0.8001649198924976], [0.8017360332663054]]
Precision for iterations:  [[0.679713796108488], [0.7523619597058836], [0.770844125379457], [0.7840314744619816], [0.7861932361841062]]
Recall for iterations:  [[0.7168513565150936], [0.7930199974525538], [0.8075404407081901], [0.8209145331804866], [0.8226977455101261]]


#### niter = 5, nepoch = 10

In [23]:
# Define 
num_nodes = 5
global_updates = 5

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 10

In [24]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'Bot_cat5_5_10.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [25]:
global_model = build_model((24,1,1))

In [26]:
optimizer = keras.optimizers.Adam(learning_rate=1e-4)

In [27]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_test, accuracy_test, f1_test, precision_test, recall_test = evaluation(global_model, xtest, ytest) 

    loss_it.append([loss_test])
    accuracy_it.append([accuracy_test])
    f1_it.append([f1_test])
    precision_it.append([precision_test])
    recall_it.append([recall_test])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/Bot_cat5_5_10.hdf5')

Epoch 1/10
4/4 [==============================] - 5s 1s/step - loss: 1.8149 - accuracy: 0.0408 - val_loss: 1.3353 - val_accuracy: 0.2263
Epoch 2/10
4/4 [==============================] - 1s 298ms/step - loss: 1.5369 - accuracy: 0.0851 - val_loss: 1.3172 - val_accuracy: 0.2147
Epoch 3/10
4/4 [==============================] - 1s 335ms/step - loss: 1.3134 - accuracy: 0.3263 - val_loss: 1.2995 - val_accuracy: 0.8316
Epoch 4/10
4/4 [==============================] - 1s 321ms/step - loss: 1.1372 - accuracy: 0.4988 - val_loss: 1.2822 - val_accuracy: 0.8674
Epoch 5/10
4/4 [==============================] - 0s 110ms/step - loss: 1.0052 - accuracy: 0.5396 - val_loss: 1.2654 - val_accuracy: 0.9168
Epoch 6/10
4/4 [==============================] - 0s 107ms/step - loss: 0.9082 - accuracy: 0.5712 - val_loss: 1.2494 - val_accuracy: 0.9189
Epoch 7/10
4/4 [==============================] - 0s 110ms/step - loss: 0.8378 - accuracy: 0.6044 - val_loss: 1.2343 - val_accuracy: 0.9189
Epoch 8/10
4/4 [=======

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
4/4 [==============================] - 4s 1s/step - loss: 0.3672 - accuracy: 0.8512 - val_loss: 2.0074 - val_accuracy: 0.7463
Epoch 2/10
4/4 [==============================] - 1s 142ms/step - loss: 0.3646 - accuracy: 0.8357 - val_loss: 2.0352 - val_accuracy: 0.7463
Epoch 3/10
4/4 [==============================] - 1s 172ms/step - loss: 0.3552 - accuracy: 0.8520 - val_loss: 2.0599 - val_accuracy: 0.7463
Epoch 4/10
4/4 [==============================] - 1s 157ms/step - loss: 0.3518 - accuracy: 0.8596 - val_loss: 2.0868 - val_accuracy: 0.7463
Epoch 5/10
4/4 [==============================] - 1s 134ms/step - loss: 0.3514 - accuracy: 0.8580 - val_loss: 2.1180 - val_accuracy: 0.7463
Epoch 6/10
4/4 [==============================] - 1s 238ms/step - loss: 0.3504 - accuracy: 0.8567 - val_loss: 2.1482 - val_accuracy: 0.7463
Epoch 7/10
4/4 [==============================] - 1s 196ms/step - loss: 0.3490 - accuracy: 0.8541 - val_loss: 2.1750 - val_accuracy: 0.7463
Epoch 8/10
4/4 [=======

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
4/4 [==============================] - 1s 275ms/step - loss: 0.3264 - accuracy: 0.8665 - val_loss: 3.3065 - val_accuracy: 0.7463
Epoch 2/10
4/4 [==============================] - 0s 119ms/step - loss: 0.3193 - accuracy: 0.8704 - val_loss: 3.3298 - val_accuracy: 0.7463
Epoch 3/10
4/4 [==============================] - 1s 129ms/step - loss: 0.3136 - accuracy: 0.8688 - val_loss: 3.3533 - val_accuracy: 0.7463
Epoch 4/10
4/4 [==============================] - 1s 129ms/step - loss: 0.3153 - accuracy: 0.8673 - val_loss: 3.3793 - val_accuracy: 0.7463
Epoch 5/10
4/4 [==============================] - 1s 201ms/step - loss: 0.3101 - accuracy: 0.8775 - val_loss: 3.4117 - val_accuracy: 0.7463
Epoch 6/10
4/4 [==============================] - 1s 197ms/step - loss: 0.3098 - accuracy: 0.8786 - val_loss: 3.4420 - val_accuracy: 0.7463
Epoch 7/10
4/4 [==============================] - 0s 98ms/step - loss: 0.3088 - accuracy: 0.8796 - val_loss: 3.4680 - val_accuracy: 0.7463
Epoch 8/10
4/4 [=====

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.7791982293128967], [0.5457212328910828], [0.7172202467918396], [0.9366443157196045], [1.1878306865692139]]
Accuracy for iterations:  [[0.7476754784584045], [0.8051203489303589], [0.8298305869102478], [0.8428225517272949], [0.8652400970458984]]
F1 for iterations:  [[0.720648941619538], [0.7835901193535014], [0.8088535623870491], [0.8221685801221861], [0.8444612759733268]]
Precision for iterations:  [[0.7651339981474904], [0.7702642411302676], [0.7927918861631134], [0.8045378779178997], [0.8254691534893449]]
Recall for iterations:  [[0.7476754553560057], [0.8051203668322506], [0.8298305948286843], [0.8428225703732009], [0.865240096802955]]


#### niter = 5, nepoch = 15

In [29]:
# Define 
num_nodes = 5
global_updates = 5

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 15

In [30]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'Bot_cat5_5_15.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [31]:
global_model = build_model((24,1,1))

In [32]:
optimizer = keras.optimizers.Adam(learning_rate=1e-4)

In [33]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_test, accuracy_test, f1_test, precision_test, recall_test = evaluation(global_model, xtest, ytest) 

    loss_it.append([loss_test])
    accuracy_it.append([accuracy_test])
    f1_it.append([f1_test])
    precision_it.append([precision_test])
    recall_it.append([recall_test])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/Bot_cat5_5_15.hdf5')

Epoch 1/15
4/4 [==============================] - 1s 336ms/step - loss: 1.5447 - accuracy: 0.1280 - val_loss: 1.4136 - val_accuracy: 0.2705
Epoch 2/15
4/4 [==============================] - 0s 109ms/step - loss: 1.3346 - accuracy: 0.3642 - val_loss: 1.4028 - val_accuracy: 0.4000
Epoch 3/15
4/4 [==============================] - 0s 91ms/step - loss: 1.1667 - accuracy: 0.5315 - val_loss: 1.3911 - val_accuracy: 0.4926
Epoch 4/15
4/4 [==============================] - 1s 130ms/step - loss: 1.0318 - accuracy: 0.5807 - val_loss: 1.3791 - val_accuracy: 0.5558
Epoch 5/15
4/4 [==============================] - 1s 146ms/step - loss: 0.9257 - accuracy: 0.6136 - val_loss: 1.3674 - val_accuracy: 0.6600
Epoch 6/15
4/4 [==============================] - 1s 193ms/step - loss: 0.8426 - accuracy: 0.6358 - val_loss: 1.3561 - val_accuracy: 0.7316
Epoch 7/15
4/4 [==============================] - 1s 148ms/step - loss: 0.7802 - accuracy: 0.6732 - val_loss: 1.3457 - val_accuracy: 0.7326
Epoch 8/15
4/4 [=====

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
4/4 [==============================] - 2s 511ms/step - loss: 0.4226 - accuracy: 0.8262 - val_loss: 0.9603 - val_accuracy: 0.7463
Epoch 2/15
4/4 [==============================] - 0s 120ms/step - loss: 0.4171 - accuracy: 0.8351 - val_loss: 0.9737 - val_accuracy: 0.7463
Epoch 3/15
4/4 [==============================] - 0s 103ms/step - loss: 0.4143 - accuracy: 0.8491 - val_loss: 0.9876 - val_accuracy: 0.7463
Epoch 4/15
4/4 [==============================] - 0s 89ms/step - loss: 0.4117 - accuracy: 0.8538 - val_loss: 1.0042 - val_accuracy: 0.7463
Epoch 5/15
4/4 [==============================] - 0s 99ms/step - loss: 0.4096 - accuracy: 0.8496 - val_loss: 1.0224 - val_accuracy: 0.7463
Epoch 6/15
4/4 [==============================] - 1s 140ms/step - loss: 0.4086 - accuracy: 0.8396 - val_loss: 1.0395 - val_accuracy: 0.7463
Epoch 7/15
4/4 [==============================] - 1s 274ms/step - loss: 0.4058 - accuracy: 0.8451 - val_loss: 1.0574 - val_accuracy: 0.7463
Epoch 8/15
4/4 [======

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
4/4 [==============================] - 3s 852ms/step - loss: 0.3740 - accuracy: 0.8483 - val_loss: 2.7542 - val_accuracy: 0.7463
Epoch 2/15
4/4 [==============================] - 1s 133ms/step - loss: 0.3664 - accuracy: 0.8501 - val_loss: 2.7868 - val_accuracy: 0.7463
Epoch 3/15
4/4 [==============================] - 1s 203ms/step - loss: 0.3620 - accuracy: 0.8551 - val_loss: 2.8099 - val_accuracy: 0.7463
Epoch 4/15
4/4 [==============================] - 1s 131ms/step - loss: 0.3602 - accuracy: 0.8623 - val_loss: 2.8444 - val_accuracy: 0.7463
Epoch 5/15
4/4 [==============================] - 1s 168ms/step - loss: 0.3606 - accuracy: 0.8615 - val_loss: 2.8786 - val_accuracy: 0.7463
Epoch 6/15
4/4 [==============================] - 1s 156ms/step - loss: 0.3587 - accuracy: 0.8575 - val_loss: 2.9089 - val_accuracy: 0.7463
Epoch 7/15
4/4 [==============================] - 0s 102ms/step - loss: 0.3571 - accuracy: 0.8575 - val_loss: 2.9370 - val_accuracy: 0.7463
Epoch 8/15
4/4 [====

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
4/4 [==============================] - 4s 900ms/step - loss: 0.3272 - accuracy: 0.8662 - val_loss: 4.5626 - val_accuracy: 0.7463
Epoch 2/15
4/4 [==============================] - 1s 135ms/step - loss: 0.3275 - accuracy: 0.8575 - val_loss: 4.5938 - val_accuracy: 0.7463
Epoch 3/15
4/4 [==============================] - 1s 169ms/step - loss: 0.3180 - accuracy: 0.8765 - val_loss: 4.6177 - val_accuracy: 0.7463
Epoch 4/15
4/4 [==============================] - 1s 165ms/step - loss: 0.3181 - accuracy: 0.8752 - val_loss: 4.6395 - val_accuracy: 0.7463
Epoch 5/15
4/4 [==============================] - 0s 110ms/step - loss: 0.3177 - accuracy: 0.8733 - val_loss: 4.6687 - val_accuracy: 0.7463
Epoch 6/15
4/4 [==============================] - 1s 192ms/step - loss: 0.3170 - accuracy: 0.8744 - val_loss: 4.6925 - val_accuracy: 0.7463
Epoch 7/15
4/4 [==============================] - 0s 122ms/step - loss: 0.3140 - accuracy: 0.8760 - val_loss: 4.7123 - val_accuracy: 0.7463
Epoch 8/15
4/4 [====

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
4/4 [==============================] - 7s 2s/step - loss: 0.2910 - accuracy: 0.8828 - val_loss: 6.3210 - val_accuracy: 0.7453
Epoch 2/15
4/4 [==============================] - 1s 308ms/step - loss: 0.2863 - accuracy: 0.8925 - val_loss: 6.3586 - val_accuracy: 0.7453
Epoch 3/15
4/4 [==============================] - 1s 211ms/step - loss: 0.2815 - accuracy: 0.9012 - val_loss: 6.3891 - val_accuracy: 0.7463
Epoch 4/15
4/4 [==============================] - 1s 168ms/step - loss: 0.2808 - accuracy: 0.8978 - val_loss: 6.4276 - val_accuracy: 0.7463
Epoch 5/15
4/4 [==============================] - 1s 174ms/step - loss: 0.2786 - accuracy: 0.9010 - val_loss: 6.4725 - val_accuracy: 0.7463
Epoch 6/15
4/4 [==============================] - 1s 180ms/step - loss: 0.2775 - accuracy: 0.9023 - val_loss: 6.5036 - val_accuracy: 0.7463
Epoch 7/15
4/4 [==============================] - 0s 121ms/step - loss: 0.2791 - accuracy: 0.8981 - val_loss: 6.5366 - val_accuracy: 0.7463
Epoch 8/15
4/4 [=======

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.6090918183326721], [0.823773980140686], [1.1313725709915161], [1.4487837553024292], [1.865320086479187]]
Accuracy for iterations:  [[0.780792236328125], [0.8341612815856934], [0.84422367811203], [0.8600178360939026], [0.880524754524231]]
F1 for iterations:  [[0.7536594069713883], [0.8133499021191904], [0.8238371708752031], [0.839970416118507], [0.8601253834170304]]
Precision for iterations:  [[0.7529922893336729], [0.7957993331606145], [0.8057311755226135], [0.8213647873320049], [0.8418270251443299]]
Recall for iterations:  [[0.780792255763597], [0.8341612533435231], [0.8442236657750605], [0.8600178321232964], [0.8805247739141511]]


#### niter = 10, nepoch = 5

In [35]:
# Define 
num_nodes = 5
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [36]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'Bot_cat5_10_5.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [37]:
global_model = build_model((24,1,1))

In [38]:
optimizer = keras.optimizers.Adam(learning_rate=1e-4)

In [39]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_test, accuracy_test, f1_test, precision_test, recall_test = evaluation(global_model, xtest, ytest) 

    loss_it.append([loss_test])
    accuracy_it.append([accuracy_test])
    f1_it.append([f1_test])
    precision_it.append([precision_test])
    recall_it.append([recall_test])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/Bot_cat5_10_5.hdf5')

Epoch 1/5
4/4 [==============================] - 3s 794ms/step - loss: 1.4631 - accuracy: 0.3274 - val_loss: 1.4056 - val_accuracy: 0.0011
Epoch 2/5
4/4 [==============================] - 1s 188ms/step - loss: 1.3614 - accuracy: 0.4440 - val_loss: 1.3994 - val_accuracy: 0.0011
Epoch 3/5
4/4 [==============================] - 1s 220ms/step - loss: 1.2646 - accuracy: 0.5093 - val_loss: 1.3929 - val_accuracy: 0.0011
Epoch 4/5
4/4 [==============================] - 1s 216ms/step - loss: 1.1744 - accuracy: 0.5802 - val_loss: 1.3860 - val_accuracy: 0.1821
Epoch 5/5
4/4 [==============================] - 1s 193ms/step - loss: 1.0913 - accuracy: 0.6081 - val_loss: 1.3784 - val_accuracy: 0.5463
Epoch 1/5
4/4 [==============================] - 2s 406ms/step - loss: 1.0005 - accuracy: 0.6121 - val_loss: 1.3698 - val_accuracy: 0.6621
Epoch 2/5
4/4 [==============================] - 1s 129ms/step - loss: 0.9275 - accuracy: 0.6350 - val_loss: 1.3618 - val_accuracy: 0.6642
Epoch 3/5
4/4 [============

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
4/4 [==============================] - 2s 454ms/step - loss: 0.5402 - accuracy: 0.7069 - val_loss: 1.1697 - val_accuracy: 0.7411
Epoch 2/5
4/4 [==============================] - 1s 142ms/step - loss: 0.5290 - accuracy: 0.7498 - val_loss: 1.1614 - val_accuracy: 0.7411
Epoch 3/5
4/4 [==============================] - 1s 141ms/step - loss: 0.5198 - accuracy: 0.7677 - val_loss: 1.1509 - val_accuracy: 0.7411
Epoch 4/5
4/4 [==============================] - 1s 129ms/step - loss: 0.5113 - accuracy: 0.7777 - val_loss: 1.1390 - val_accuracy: 0.7421
Epoch 5/5
4/4 [==============================] - 1s 133ms/step - loss: 0.5056 - accuracy: 0.7864 - val_loss: 1.1271 - val_accuracy: 0.7442
Epoch 1/5
4/4 [==============================] - 3s 690ms/step - loss: 0.5050 - accuracy: 0.7498 - val_loss: 1.1283 - val_accuracy: 0.7411
Epoch 2/5
4/4 [==============================] - 1s 177ms/step - loss: 0.4966 - accuracy: 0.7627 - val_loss: 1.1140 - val_accuracy: 0.7411
Epoch 3/5
4/4 [============

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
4/4 [==============================] - 1s 262ms/step - loss: 0.4588 - accuracy: 0.8120 - val_loss: 0.8730 - val_accuracy: 0.7442
Epoch 2/5
4/4 [==============================] - 0s 83ms/step - loss: 0.4527 - accuracy: 0.8009 - val_loss: 0.8716 - val_accuracy: 0.7442
Epoch 3/5
4/4 [==============================] - 0s 87ms/step - loss: 0.4487 - accuracy: 0.8214 - val_loss: 0.8691 - val_accuracy: 0.7442
Epoch 4/5
4/4 [==============================] - 0s 87ms/step - loss: 0.4463 - accuracy: 0.8280 - val_loss: 0.8659 - val_accuracy: 0.7442
Epoch 5/5
4/4 [==============================] - 0s 87ms/step - loss: 0.4430 - accuracy: 0.8233 - val_loss: 0.8635 - val_accuracy: 0.7442
Epoch 1/5
4/4 [==============================] - 2s 576ms/step - loss: 0.4525 - accuracy: 0.8014 - val_loss: 0.9130 - val_accuracy: 0.7463
Epoch 2/5
4/4 [==============================] - 1s 135ms/step - loss: 0.4473 - accuracy: 0.8164 - val_loss: 0.9082 - val_accuracy: 0.7463
Epoch 3/5
4/4 [================

...

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
4/4 [==============================] - 1s 199ms/step - loss: 0.2895 - accuracy: 0.8820 - val_loss: 5.0538 - val_accuracy: 0.7463
Epoch 2/5
4/4 [==============================] - 0s 83ms/step - loss: 0.2858 - accuracy: 0.8931 - val_loss: 5.0803 - val_accuracy: 0.7463
Epoch 3/5
4/4 [==============================] - 0s 101ms/step - loss: 0.2810 - accuracy: 0.8928 - val_loss: 5.1106 - val_accuracy: 0.7463
Epoch 4/5
4/4 [==============================] - 0s 83ms/step - loss: 0.2789 - accuracy: 0.8957 - val_loss: 5.1513 - val_accuracy: 0.7463
Epoch 5/5
4/4 [==============================] - 0s 95ms/step - loss: 0.2768 - accuracy: 0.8954 - val_loss: 5.1953 - val_accuracy: 0.7463
Epoch 1/5
4/4 [==============================] - 1s 223ms/step - loss: 0.2926 - accuracy: 0.8854 - val_loss: 5.6612 - val_accuracy: 0.7463
Epoch 2/5
4/4 [==============================] - 0s 75ms/step - loss: 0.2861 - accuracy: 0.8860 - val_loss: 5.6840 - val_accuracy: 0.7463
Epoch 3/5
4/4 [================

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
4/4 [==============================] - 0s 110ms/step - loss: 0.2674 - accuracy: 0.8999 - val_loss: 5.9012 - val_accuracy: 0.7463
Epoch 2/5
4/4 [==============================] - 0s 55ms/step - loss: 0.2637 - accuracy: 0.9012 - val_loss: 5.9346 - val_accuracy: 0.7463
Epoch 3/5
4/4 [==============================] - 0s 86ms/step - loss: 0.2607 - accuracy: 0.9039 - val_loss: 5.9681 - val_accuracy: 0.7463
Epoch 4/5
4/4 [==============================] - 0s 83ms/step - loss: 0.2575 - accuracy: 0.9070 - val_loss: 6.0054 - val_accuracy: 0.7463
Epoch 5/5
4/4 [==============================] - 0s 51ms/step - loss: 0.2574 - accuracy: 0.9044 - val_loss: 6.0424 - val_accuracy: 0.7463
Epoch 1/5
4/4 [==============================] - 1s 225ms/step - loss: 0.2695 - accuracy: 0.8999 - val_loss: 6.5616 - val_accuracy: 0.7463
Epoch 2/5
4/4 [==============================] - 0s 55ms/step - loss: 0.2678 - accuracy: 0.8981 - val_loss: 6.5788 - val_accuracy: 0.7463
Epoch 3/5
4/4 [=================

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [40]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[1.1550443172454834], [0.8250648975372314], [0.6484357118606567], [0.6986439228057861], [0.8372128009796143], [0.9662157893180847], [1.0876147747039795], [1.2319685220718384], [1.3788150548934937], [1.5280158519744873]]
Accuracy for iterations:  [[0.6918863654136658], [0.7913641333580017], [0.8052477240562439], [0.806394100189209], [0.825245201587677], [0.843586802482605], [0.8521207571029663], [0.861801028251648], [0.8700802326202393], [0.8737740516662598]]
F1 for iterations:  [[0.663905625937127], [0.770006084722125], [0.786549714983623], [0.7877943804697545], [0.8058078354121923], [0.8235540904424847], [0.8321073051618739], [0.8417053917615172], [0.8503681519956461], [0.8538601282642442]]
Precision for iterations:  [[0.6559311434515549], [0.7577770812953851], [0.7690638028615107], [0.7708101115474214], [0.7875128161477374], [0.8049030050878835], [0.8135102196995915], [0.8230668752168386], [0.8339978699731341], [0.8368877896280794]]
Recall for iterations:  [[0.

#### niter = 15, nepoch = 5

In [41]:
# Define 
num_nodes = 5
global_updates = 15

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [42]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'Bot_cat5_15_5.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [43]:
global_model = build_model((24,1,1))

In [44]:
optimizer = keras.optimizers.Adam(learning_rate=1e-4)

In [45]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_test, accuracy_test, f1_test, precision_test, recall_test = evaluation(global_model, xtest, ytest) 

    loss_it.append([loss_test])
    accuracy_it.append([accuracy_test])
    f1_it.append([f1_test])
    precision_it.append([precision_test])
    recall_it.append([recall_test])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/Bot_cat5_15_5.hdf5')

Epoch 1/5
4/4 [==============================] - 1s 258ms/step - loss: 1.5043 - accuracy: 0.2794 - val_loss: 1.3803 - val_accuracy: 0.2337
Epoch 2/5
4/4 [==============================] - 0s 75ms/step - loss: 1.3681 - accuracy: 0.4235 - val_loss: 1.3736 - val_accuracy: 0.3432
Epoch 3/5
4/4 [==============================] - 0s 75ms/step - loss: 1.2559 - accuracy: 0.4967 - val_loss: 1.3664 - val_accuracy: 0.4905
Epoch 4/5
4/4 [==============================] - 0s 83ms/step - loss: 1.1537 - accuracy: 0.5544 - val_loss: 1.3587 - val_accuracy: 0.6116
Epoch 5/5
4/4 [==============================] - 0s 79ms/step - loss: 1.0609 - accuracy: 0.5765 - val_loss: 1.3502 - val_accuracy: 0.7505
Epoch 1/5
4/4 [==============================] - 1s 172ms/step - loss: 0.9923 - accuracy: 0.5818 - val_loss: 1.3417 - val_accuracy: 0.7800
Epoch 2/5
4/4 [==============================] - 1s 130ms/step - loss: 0.9049 - accuracy: 0.5952 - val_loss: 1.3313 - val_accuracy: 0.7842
Epoch 3/5
4/4 [================

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
4/4 [==============================] - 1s 265ms/step - loss: 0.3449 - accuracy: 0.8567 - val_loss: 3.1582 - val_accuracy: 0.7463
Epoch 2/5
4/4 [==============================] - 0s 52ms/step - loss: 0.3415 - accuracy: 0.8615 - val_loss: 3.2051 - val_accuracy: 0.7463
Epoch 3/5
4/4 [==============================] - 0s 102ms/step - loss: 0.3363 - accuracy: 0.8580 - val_loss: 3.2197 - val_accuracy: 0.7463
Epoch 4/5
4/4 [==============================] - 0s 59ms/step - loss: 0.3342 - accuracy: 0.8623 - val_loss: 3.2443 - val_accuracy: 0.7463
Epoch 5/5
4/4 [==============================] - 0s 53ms/step - loss: 0.3313 - accuracy: 0.8609 - val_loss: 3.2763 - val_accuracy: 0.7463
Epoch 1/5
4/4 [==============================] - 1s 314ms/step - loss: 0.3493 - accuracy: 0.8475 - val_loss: 3.6875 - val_accuracy: 0.7463
Epoch 2/5
4/4 [==============================] - 0s 114ms/step - loss: 0.3464 - accuracy: 0.8525 - val_loss: 3.7179 - val_accuracy: 0.7463
Epoch 3/5
4/4 [===============

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
4/4 [==============================] - 1s 175ms/step - loss: 0.3234 - accuracy: 0.8591 - val_loss: 4.0388 - val_accuracy: 0.7463
Epoch 2/5
4/4 [==============================] - 0s 104ms/step - loss: 0.3170 - accuracy: 0.8760 - val_loss: 4.0743 - val_accuracy: 0.7463
Epoch 3/5
4/4 [==============================] - 0s 67ms/step - loss: 0.3117 - accuracy: 0.8799 - val_loss: 4.0934 - val_accuracy: 0.7463
Epoch 4/5
4/4 [==============================] - 0s 58ms/step - loss: 0.3084 - accuracy: 0.8725 - val_loss: 4.1143 - val_accuracy: 0.7463
Epoch 5/5
4/4 [==============================] - 0s 92ms/step - loss: 0.3080 - accuracy: 0.8744 - val_loss: 4.1425 - val_accuracy: 0.7463
Epoch 1/5
4/4 [==============================] - 1s 267ms/step - loss: 0.3248 - accuracy: 0.8575 - val_loss: 4.6206 - val_accuracy: 0.7463
Epoch 2/5
4/4 [==============================] - 0s 54ms/step - loss: 0.3225 - accuracy: 0.8591 - val_loss: 4.6273 - val_accuracy: 0.7463
Epoch 3/5
4/4 [================

...

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
4/4 [==============================] - 1s 169ms/step - loss: 0.2256 - accuracy: 0.9181 - val_loss: 9.8179 - val_accuracy: 0.7463
Epoch 2/5
4/4 [==============================] - 0s 79ms/step - loss: 0.2194 - accuracy: 0.9223 - val_loss: 9.8081 - val_accuracy: 0.7463
Epoch 3/5
4/4 [==============================] - 0s 75ms/step - loss: 0.2166 - accuracy: 0.9257 - val_loss: 9.8356 - val_accuracy: 0.7463
Epoch 4/5
4/4 [==============================] - 0s 63ms/step - loss: 0.2154 - accuracy: 0.9228 - val_loss: 9.9033 - val_accuracy: 0.7463
Epoch 5/5
4/4 [==============================] - 0s 71ms/step - loss: 0.2153 - accuracy: 0.9234 - val_loss: 9.9578 - val_accuracy: 0.7463
Epoch 1/5
4/4 [==============================] - 1s 176ms/step - loss: 0.2349 - accuracy: 0.9065 - val_loss: 10.9756 - val_accuracy: 0.7463
Epoch 2/5
4/4 [==============================] - 0s 71ms/step - loss: 0.2322 - accuracy: 0.9157 - val_loss: 11.0220 - val_accuracy: 0.7463
Epoch 3/5
4/4 [===============

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
4/4 [==============================] - 1s 130ms/step - loss: 0.2132 - accuracy: 0.9247 - val_loss: 10.9540 - val_accuracy: 0.7463
Epoch 2/5
4/4 [==============================] - 0s 63ms/step - loss: 0.2126 - accuracy: 0.9263 - val_loss: 10.9338 - val_accuracy: 0.7463
Epoch 3/5
4/4 [==============================] - 0s 63ms/step - loss: 0.2053 - accuracy: 0.9302 - val_loss: 10.9518 - val_accuracy: 0.7463
Epoch 4/5
4/4 [==============================] - 0s 107ms/step - loss: 0.2030 - accuracy: 0.9286 - val_loss: 11.0361 - val_accuracy: 0.7463
Epoch 5/5
4/4 [==============================] - 0s 60ms/step - loss: 0.2011 - accuracy: 0.9268 - val_loss: 11.1049 - val_accuracy: 0.7463
Epoch 1/5
4/4 [==============================] - 1s 241ms/step - loss: 0.2311 - accuracy: 0.9120 - val_loss: 12.1993 - val_accuracy: 0.7463
Epoch 2/5
4/4 [==============================] - 0s 64ms/step - loss: 0.2173 - accuracy: 0.9220 - val_loss: 12.2226 - val_accuracy: 0.7463
Epoch 3/5
4/4 [=========

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [46]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[1.0596972703933716], [0.7573782801628113], [0.6128680109977722], [0.655382513999939], [0.7731356620788574], [0.9085248708724976], [1.0485225915908813], [1.1833363771438599], [1.3473787307739258], [1.513008713722229], [1.6888782978057861], [1.9132239818572998], [2.127502679824829], [2.331770181655884], [2.5594594478607178]]
Accuracy for iterations:  [[0.7185071706771851], [0.7787542939186096], [0.8042287826538086], [0.8214240074157715], [0.8308495879173279], [0.8342885971069336], [0.8398929834365845], [0.8494459390640259], [0.8601452112197876], [0.8696981072425842], [0.8753024935722351], [0.8758119940757751], [0.881798505783081], [0.8903324604034424], [0.8867660164833069]]
F1 for iterations:  [[0.7066118606867507], [0.7552084874456257], [0.7810820546510275], [0.7998218161642641], [0.8093954028035438], [0.8127504591206204], [0.8183607764228779], [0.8279863339390258], [0.8402797547354717], [0.8497096179882804], [0.8549976481348485], [0.8561253795613647], [0.8621113

#### niter = 10, nepoch = 10

In [47]:
# Define 
num_nodes = 5
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 10

In [48]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'Bot_cat5_10_10.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [49]:
global_model = build_model((24,1,1))

In [50]:
optimizer = keras.optimizers.Adam(learning_rate=1e-4)

In [51]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_test, accuracy_test, f1_test, precision_test, recall_test = evaluation(global_model, xtest, ytest) 

    loss_it.append([loss_test])
    accuracy_it.append([accuracy_test])
    f1_it.append([f1_test])
    precision_it.append([precision_test])
    recall_it.append([recall_test])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/Bot_cat5_10_10.hdf5')

Epoch 1/10
4/4 [==============================] - 1s 280ms/step - loss: 1.5874 - accuracy: 0.2863 - val_loss: 1.4545 - val_accuracy: 0.0000e+00
Epoch 2/10
4/4 [==============================] - 0s 67ms/step - loss: 1.3905 - accuracy: 0.3292 - val_loss: 1.4357 - val_accuracy: 0.0000e+00
Epoch 3/10
4/4 [==============================] - 0s 63ms/step - loss: 1.2287 - accuracy: 0.3769 - val_loss: 1.4176 - val_accuracy: 0.0000e+00
Epoch 4/10
4/4 [==============================] - 0s 62ms/step - loss: 1.0959 - accuracy: 0.5075 - val_loss: 1.4000 - val_accuracy: 0.0000e+00
Epoch 5/10
4/4 [==============================] - 0s 67ms/step - loss: 0.9869 - accuracy: 0.5928 - val_loss: 1.3827 - val_accuracy: 0.0074
Epoch 6/10
4/4 [==============================] - 1s 132ms/step - loss: 0.9013 - accuracy: 0.6431 - val_loss: 1.3658 - val_accuracy: 0.2747
Epoch 7/10
4/4 [==============================] - 0s 60ms/step - loss: 0.8316 - accuracy: 0.6734 - val_loss: 1.3499 - val_accuracy: 0.6189
Epoch 8/1

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
4/4 [==============================] - 1s 315ms/step - loss: 0.4602 - accuracy: 0.8027 - val_loss: 0.7803 - val_accuracy: 0.7463
Epoch 2/10
4/4 [==============================] - 0s 87ms/step - loss: 0.4548 - accuracy: 0.8075 - val_loss: 0.7812 - val_accuracy: 0.7463
Epoch 3/10
4/4 [==============================] - 0s 99ms/step - loss: 0.4518 - accuracy: 0.8214 - val_loss: 0.7817 - val_accuracy: 0.7463
Epoch 4/10
4/4 [==============================] - 0s 63ms/step - loss: 0.4499 - accuracy: 0.8262 - val_loss: 0.7816 - val_accuracy: 0.7463
Epoch 5/10
4/4 [==============================] - 0s 59ms/step - loss: 0.4485 - accuracy: 0.8238 - val_loss: 0.7824 - val_accuracy: 0.7463
Epoch 6/10
4/4 [==============================] - 0s 56ms/step - loss: 0.4448 - accuracy: 0.8259 - val_loss: 0.7847 - val_accuracy: 0.7463
Epoch 7/10
4/4 [==============================] - 0s 94ms/step - loss: 0.4420 - accuracy: 0.8307 - val_loss: 0.7869 - val_accuracy: 0.7463
Epoch 8/10
4/4 [==========

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
4/4 [==============================] - 1s 172ms/step - loss: 0.4076 - accuracy: 0.8383 - val_loss: 1.7967 - val_accuracy: 0.7463
Epoch 2/10
4/4 [==============================] - 0s 79ms/step - loss: 0.4005 - accuracy: 0.8378 - val_loss: 1.8426 - val_accuracy: 0.7463
Epoch 3/10
4/4 [==============================] - 0s 55ms/step - loss: 0.3989 - accuracy: 0.8501 - val_loss: 1.8939 - val_accuracy: 0.7463
Epoch 4/10
4/4 [==============================] - 0s 93ms/step - loss: 0.3958 - accuracy: 0.8588 - val_loss: 1.9429 - val_accuracy: 0.7463
Epoch 5/10
4/4 [==============================] - 0s 103ms/step - loss: 0.3962 - accuracy: 0.8509 - val_loss: 1.9898 - val_accuracy: 0.7463
Epoch 6/10
4/4 [==============================] - 0s 63ms/step - loss: 0.3949 - accuracy: 0.8470 - val_loss: 2.0368 - val_accuracy: 0.7463
Epoch 7/10
4/4 [==============================] - 0s 91ms/step - loss: 0.3933 - accuracy: 0.8520 - val_loss: 2.0830 - val_accuracy: 0.7463
Epoch 8/10
4/4 [=========

...

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
4/4 [==============================] - 1s 274ms/step - loss: 0.1762 - accuracy: 0.9479 - val_loss: 17.8410 - val_accuracy: 0.7463
Epoch 2/10
4/4 [==============================] - 0s 89ms/step - loss: 0.1687 - accuracy: 0.9547 - val_loss: 17.8794 - val_accuracy: 0.7463
Epoch 3/10
4/4 [==============================] - 0s 97ms/step - loss: 0.1681 - accuracy: 0.9529 - val_loss: 17.9822 - val_accuracy: 0.7463
Epoch 4/10
4/4 [==============================] - 0s 87ms/step - loss: 0.1661 - accuracy: 0.9555 - val_loss: 18.0649 - val_accuracy: 0.7463
Epoch 5/10
4/4 [==============================] - 0s 107ms/step - loss: 0.1657 - accuracy: 0.9552 - val_loss: 18.1276 - val_accuracy: 0.7463
Epoch 6/10
4/4 [==============================] - 0s 122ms/step - loss: 0.1646 - accuracy: 0.9558 - val_loss: 18.2166 - val_accuracy: 0.7463s - loss: 0.1668 - accuracy: 0.
Epoch 7/10
4/4 [==============================] - 0s 67ms/step - loss: 0.1632 - accuracy: 0.9600 - val_loss: 18.2684 - val_acc

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
4/4 [==============================] - 1s 252ms/step - loss: 0.1521 - accuracy: 0.9605 - val_loss: 20.8945 - val_accuracy: 0.7463
Epoch 2/10
4/4 [==============================] - ETA: 0s - loss: 0.1435 - accuracy: 0.96 - 0s 91ms/step - loss: 0.1423 - accuracy: 0.9687 - val_loss: 20.8903 - val_accuracy: 0.7463
Epoch 3/10
4/4 [==============================] - 0s 103ms/step - loss: 0.1433 - accuracy: 0.9668 - val_loss: 20.9698 - val_accuracy: 0.7463
Epoch 4/10
4/4 [==============================] - 0s 67ms/step - loss: 0.1375 - accuracy: 0.9700 - val_loss: 21.0623 - val_accuracy: 0.7463
Epoch 5/10
4/4 [==============================] - 0s 68ms/step - loss: 0.1380 - accuracy: 0.9734 - val_loss: 21.1093 - val_accuracy: 0.7463
Epoch 6/10
4/4 [==============================] - 0s 54ms/step - loss: 0.1363 - accuracy: 0.9716 - val_loss: 21.1643 - val_accuracy: 0.7463
Epoch 7/10
4/4 [==============================] - 0s 55ms/step - loss: 0.1350 - accuracy: 0.9716 - val_loss: 21.2316

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [52]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.741174578666687], [0.721889078617096], [1.1435155868530273], [1.5367358922958374], [1.9824142456054688], [2.5163683891296387], [3.0709986686706543], [3.6691699028015137], [4.246504306793213], [4.737659454345703]]
Accuracy for iterations:  [[0.7722582817077637], [0.8235893249511719], [0.8359444737434387], [0.8447331786155701], [0.8638390302658081], [0.881289005279541], [0.8866386413574219], [0.8840911984443665], [0.8839638233184814], [0.8881670832633972]]
F1 for iterations:  [[0.752678968212795], [0.8023166100457407], [0.8149571501771576], [0.8247766681076512], [0.8444316054296007], [0.8615941280496088], [0.866910366299672], [0.8645342796227383], [0.8644119035705986], [0.8685353347906517]]
Precision for iterations:  [[0.7369072015188517], [0.7861055870587029], [0.7985382871058048], [0.806516977336291], [0.8288643452679282], [0.8500741922001813], [0.8581963120151107], [0.861980880363312], [0.864535139033998], [0.8677590098811758]]
Recall for iterations:  [[0.772

#### niter = 15, nepoch = 15

In [53]:
# Define 
num_nodes = 5
global_updates = 15

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 15

In [54]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'Bot_cat5_15_15.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [55]:
global_model = build_model((24,1,1))

In [56]:
optimizer = keras.optimizers.Adam(learning_rate=1e-4)

In [57]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_test, accuracy_test, f1_test, precision_test, recall_test = evaluation(global_model, xtest, ytest) 

    loss_it.append([loss_test])
    accuracy_it.append([accuracy_test])
    f1_it.append([f1_test])
    precision_it.append([precision_test])
    recall_it.append([recall_test])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/Bot_cat5_15_15.hdf5')

Epoch 1/15
4/4 [==============================] - 1s 180ms/step - loss: 1.2551 - accuracy: 0.3958 - val_loss: 1.4166 - val_accuracy: 0.0011
Epoch 2/15
4/4 [==============================] - 1s 130ms/step - loss: 1.1365 - accuracy: 0.4490 - val_loss: 1.4086 - val_accuracy: 0.0042
Epoch 3/15
4/4 [==============================] - 0s 91ms/step - loss: 1.0374 - accuracy: 0.5320 - val_loss: 1.3995 - val_accuracy: 0.2716
Epoch 4/15
4/4 [==============================] - 0s 76ms/step - loss: 0.9544 - accuracy: 0.5712 - val_loss: 1.3901 - val_accuracy: 0.4379
Epoch 5/15
4/4 [==============================] - 0s 87ms/step - loss: 0.8840 - accuracy: 0.6300 - val_loss: 1.3805 - val_accuracy: 0.6284
Epoch 6/15
4/4 [==============================] - 0s 67ms/step - loss: 0.8261 - accuracy: 0.6405 - val_loss: 1.3707 - val_accuracy: 0.6979
Epoch 7/15
4/4 [==============================] - 0s 79ms/step - loss: 0.7764 - accuracy: 0.6408 - val_loss: 1.3612 - val_accuracy: 0.7116
Epoch 8/15
4/4 [=========

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
4/4 [==============================] - 1s 201ms/step - loss: 0.4137 - accuracy: 0.8370 - val_loss: 1.0195 - val_accuracy: 0.7432
Epoch 2/15
4/4 [==============================] - 0s 75ms/step - loss: 0.4112 - accuracy: 0.8138 - val_loss: 1.0410 - val_accuracy: 0.7432
Epoch 3/15
4/4 [==============================] - 0s 99ms/step - loss: 0.4038 - accuracy: 0.8459 - val_loss: 1.0649 - val_accuracy: 0.7411
Epoch 4/15
4/4 [==============================] - 0s 95ms/step - loss: 0.4020 - accuracy: 0.8520 - val_loss: 1.0907 - val_accuracy: 0.7411
Epoch 5/15
4/4 [==============================] - 0s 99ms/step - loss: 0.4001 - accuracy: 0.8515 - val_loss: 1.1188 - val_accuracy: 0.7411
Epoch 6/15
4/4 [==============================] - 0s 87ms/step - loss: 0.3972 - accuracy: 0.8536 - val_loss: 1.1470 - val_accuracy: 0.7411
Epoch 7/15
4/4 [==============================] - 0s 91ms/step - loss: 0.3950 - accuracy: 0.8525 - val_loss: 1.1745 - val_accuracy: 0.7411
Epoch 8/15
4/4 [==========

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
4/4 [==============================] - 0s 113ms/step - loss: 0.3695 - accuracy: 0.8457 - val_loss: 3.1099 - val_accuracy: 0.7463
Epoch 2/15
4/4 [==============================] - 0s 107ms/step - loss: 0.3573 - accuracy: 0.8523 - val_loss: 3.1678 - val_accuracy: 0.7463
Epoch 3/15
4/4 [==============================] - 0s 63ms/step - loss: 0.3527 - accuracy: 0.8509 - val_loss: 3.2139 - val_accuracy: 0.7463
Epoch 4/15
4/4 [==============================] - 0s 81ms/step - loss: 0.3484 - accuracy: 0.8588 - val_loss: 3.2626 - val_accuracy: 0.7463
Epoch 5/15
4/4 [==============================] - 0s 103ms/step - loss: 0.3487 - accuracy: 0.8599 - val_loss: 3.3132 - val_accuracy: 0.7463
Epoch 6/15
4/4 [==============================] - 0s 99ms/step - loss: 0.3453 - accuracy: 0.8578 - val_loss: 3.3702 - val_accuracy: 0.7463
Epoch 7/15
4/4 [==============================] - 0s 97ms/step - loss: 0.3450 - accuracy: 0.8586 - val_loss: 3.4190 - val_accuracy: 0.7463
Epoch 8/15
4/4 [========

...

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
4/4 [==============================] - 1s 217ms/step - loss: 0.0791 - accuracy: 0.9866 - val_loss: 41.5576 - val_accuracy: 0.7463
Epoch 2/15
4/4 [==============================] - 0s 111ms/step - loss: 0.0711 - accuracy: 0.9910 - val_loss: 41.5393 - val_accuracy: 0.7463
Epoch 3/15
4/4 [==============================] - 0s 109ms/step - loss: 0.0599 - accuracy: 0.9963 - val_loss: 41.4201 - val_accuracy: 0.7463
Epoch 4/15
4/4 [==============================] - 0s 105ms/step - loss: 0.0633 - accuracy: 0.9937 - val_loss: 41.5651 - val_accuracy: 0.7463
Epoch 5/15
4/4 [==============================] - 0s 91ms/step - loss: 0.0618 - accuracy: 0.9958 - val_loss: 41.7683 - val_accuracy: 0.7463
Epoch 6/15
4/4 [==============================] - 0s 103ms/step - loss: 0.0581 - accuracy: 0.9950 - val_loss: 41.7689 - val_accuracy: 0.7463
Epoch 7/15
4/4 [==============================] - 0s 91ms/step - loss: 0.0577 - accuracy: 0.9947 - val_loss: 41.7829 - val_accuracy: 0.7463
Epoch 8/15
4/4 

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
4/4 [==============================] - 1s 304ms/step - loss: 0.0582 - accuracy: 0.9947 - val_loss: 45.9729 - val_accuracy: 0.7463
Epoch 2/15
4/4 [==============================] - 1s 131ms/step - loss: 0.0658 - accuracy: 0.9905 - val_loss: 45.8674 - val_accuracy: 0.7463
Epoch 3/15
4/4 [==============================] - 1s 131ms/step - loss: 0.0485 - accuracy: 0.9960 - val_loss: 45.7885 - val_accuracy: 0.7463
Epoch 4/15
4/4 [==============================] - 0s 107ms/step - loss: 0.0565 - accuracy: 0.9960 - val_loss: 46.0345 - val_accuracy: 0.7463
Epoch 5/15
4/4 [==============================] - 0s 110ms/step - loss: 0.0453 - accuracy: 0.9976 - val_loss: 46.2028 - val_accuracy: 0.7463
Epoch 6/15
4/4 [==============================] - 0s 95ms/step - loss: 0.0465 - accuracy: 0.9963 - val_loss: 46.2556 - val_accuracy: 0.7463
Epoch 7/15
4/4 [==============================] - 0s 102ms/step - loss: 0.0452 - accuracy: 0.9976 - val_loss: 46.2692 - val_accuracy: 0.7463
Epoch 8/15
4/4

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [58]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.6459775567054749], [0.9217336773872375], [1.3213547468185425], [1.718051552772522], [2.2361392974853516], [2.8254101276397705], [3.4649147987365723], [4.154364109039307], [4.9342427253723145], [5.702364444732666], [6.452792644500732], [7.314422607421875], [8.126568794250488], [8.957623481750488], [9.917572021484375]]
Accuracy for iterations:  [[0.7942937016487122], [0.8280473947525024], [0.8454973697662354], [0.863074779510498], [0.8839638233184814], [0.8960642218589783], [0.8981021642684937], [0.90676349401474], [0.9150426983833313], [0.9174627661705017], [0.8902050852775574], [0.8923704028129578], [0.8833269476890564], [0.8801426291465759], [0.8477900624275208]]
F1 for iterations:  [[0.7736697785178716], [0.8085999760118033], [0.8247585318938587], [0.8419395383603587], [0.8630786415106072], [0.8753949198511406], [0.8777792414086989], [0.8862742545685637], [0.8943456146588599], [0.8967012025502125], [0.8705297037698488], [0.8726619019259447], [0.8637969772365